In [7]:
from math import sqrt
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

from QOptCraft.basis import get_photon_basis
from QOptCraft.state import PureState, MixedState, Fock
from QOptCraft.invariant import (
    can_transition,
    can_transition_basis,
    photon_invariant,
    photon_invariant_basis,
)

Define the cost function to optimize and the restrictions to fullfil.

In [13]:
FOCK_LIST = tuple(fock for fock in get_photon_basis(modes=6, photons=4) if fock[-1] != 1 and fock[-2] != 1)
len_fock = len(FOCK_LIST)

def invariant(x: NDArray) -> float:
    """Generate a constraint to make the invariant equal to zero.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: sum of square differences between in and out invariants
            for all the input and output states.
    """
    fock_in = [(1, 0, 1, 0, 1, 1), (1, 0, 0, 1, 1, 1), (0, 1, 1, 0, 1, 1), (0, 1, 0, 1, 1, 1)]
    try:
        AMP = sqrt(1 - x @ x)
    except ValueError as e:
        # AMP = sqrt(x @ x - 1)
        AMP = 0
        # raise e

    invariant_constraint = 0

    for _ in range(4):
        coefs_in = np.sqrt(np.random.uniform(0, 1, 4)) * np.exp(1.j * np.random.uniform(0, 2 * np.pi, 4))
        coefs_in = coefs_in / np.sum(np.abs(coefs_in)**2)
        state_in = PureState(fock_in, coefs_in)
        inv_in = photon_invariant(state_in)

        coefs_out = AMP * np.diag([1,1,1,-1]) @ coefs_in
        state_out = PureState([*fock_in, *FOCK_LIST], np.append(coefs_out, x))
        inv_out = photon_invariant(state_out)
        invariant_constraint += (inv_in - inv_out) ** 2

    print(f"prob = {1 - x @ x}")
    print(f"Invariant = {invariant_constraint}\n")

    return invariant_constraint


def cost(x: NDArray) -> float:
    """Cost function to minimize. Its value is minus the probability
    of measuring the ancilla 11.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: minus the probability.
    """
    prob = 1 - x @ x
    print(f"{prob = }")
    print(f"Invariant = {invariant(x)}\n")
    return -prob

Now we define the constraints of our optimizer. Since the probabilities must be between 0 and 1:

In [14]:
x_init = np.array([1 / np.sqrt(len_fock + 3)] * (len_fock))
method = "SLSQP"
constr_prob = {"type": "ineq", "fun": lambda x: 1 - x @ x}
(res := minimize(invariant, x_init, method=method, constraints=constr_prob))

prob = 0.0434782608695653
Invariant = 23.21297870322284

prob = 0.043478257281787114
Invariant = 24.17939706548242

prob = 0.043478257281787114
Invariant = 24.04513762764531

prob = 0.043478257281787114
Invariant = 23.542242577256935

prob = 0.043478257281787114
Invariant = 22.39547423524865

prob = 0.043478257281787114
Invariant = 24.18268465985626

prob = 0.043478257281787114
Invariant = 24.652409374259058

prob = 0.043478257281787114
Invariant = 22.13125553994692

prob = 0.043478257281787114
Invariant = 25.41423452421815

prob = 0.043478257281787114
Invariant = 24.79447944478075

prob = 0.043478257281787114
Invariant = 22.198487635221507

prob = 0.043478257281787114
Invariant = 23.73210387561424

prob = 0.043478257281787114
Invariant = 23.005552577797005

prob = 0.043478257281787114
Invariant = 21.384771047623154

prob = 0.043478257281787114
Invariant = 22.568640588362776

prob = 0.043478257281787114
Invariant = 24.31345079167435

prob = 0.043478257281787114
Invariant = 24.224242530

KeyboardInterrupt: 

In [16]:
x_init = np.array([1 / np.sqrt(len_fock + 3)] * (len_fock))
method = "COBYLA"
constr_prob = {"type": "ineq", "fun": lambda x: 1 - x @ x}
(res := minimize(invariant, x_init, method=method, constraints=[constr_prob]))

prob = 0.0434782608695653
Invariant = 24.955731669542125

prob = -1.1972934453019732
Invariant = 1.0169517989689434

prob = -2.438065151473512
Invariant = 0.06157130244084216

prob = -3.6788368576450505
Invariant = 0.0830072664728046

prob = -3.6788368576450505
Invariant = 0.03726021369961191

prob = -4.919608563816589
Invariant = 0.10422825450705192

prob = -4.919608563816588
Invariant = 0.05269953194540413

prob = -4.919608563816589
Invariant = 0.006517077393695981

prob = -6.160380269988128
Invariant = 0.019888048099712896

prob = -6.160380269988128
Invariant = 0.031070980293803174

prob = -6.160380269988128
Invariant = 0.038549705602690734

prob = -6.160380269988128
Invariant = 0.023537799170942914

prob = -6.160380269988128
Invariant = 0.10162532023307402

prob = -6.160380269988128
Invariant = 0.027420949962625896

prob = -6.160380269988128
Invariant = 0.004604492987479518

prob = -7.401151976159664
Invariant = 0.027744420717001162

prob = -7.401151976159664
Invariant = 0.00338231

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.9936072265374732
       x: [ 3.480e-01  3.781e-01 ...  5.893e-02  2.940e-02]
    nfev: 877
   maxcv: 0.0

In [4]:
x_init = np.array([1 / np.sqrt(len_fock + 3)] * (len_fock))
method = "trust-constr"
constr_prob = NonlinearConstraint(lambda x: x @ x, 0, 1)
(res := minimize(invariant, x_init, method=method, constraints=constr_prob))

prob = 0.0434782608695653
Invariant = 23.397618966178975

prob = 0.043478257281787114
Invariant = 23.973009259179946

prob = 0.043478257281787114
Invariant = 22.108347266227295

prob = 0.043478257281787114
Invariant = 24.792191166496664

prob = 0.043478257281787114
Invariant = 23.431655624415203

prob = 0.043478257281787114
Invariant = 25.214186702626844

prob = 0.043478257281787114
Invariant = 25.247089600365644

prob = 0.043478257281787114
Invariant = 22.622774561067793

prob = 0.043478257281787114
Invariant = 24.343073490508996

prob = 0.043478257281787114
Invariant = 23.327323534967974

prob = 0.043478257281787114
Invariant = 24.145232598571464

prob = 0.043478257281787114
Invariant = 23.92304706705451

prob = 0.043478257281787114
Invariant = 23.256264850866717

prob = 0.043478257281787114
Invariant = 24.007342565658508

prob = 0.043478257281787114
Invariant = 25.174939955687613

prob = 0.043478257281787114
Invariant = 23.769373192597254

prob = 0.043478257281787114
Invariant = 25.

           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: 0.002114364104194367
                 x: [ 1.359e-01  1.871e-01 ...  5.110e-02  9.754e-02]
               nit: 373
              nfev: 41808
              njev: 624
              nhev: 0
          cg_niter: 5881
      cg_stop_cond: 2
              grad: [ 1.029e+06  4.669e+06 ...  4.879e+06  2.205e+06]
   lagrangian_grad: [-1.178e+06  1.630e+06 ...  4.049e+06  6.206e+05]
            constr: [array([ 9.913e-01])]
               jac: [array([[ 2.718e-01,  3.742e-01, ...,  1.022e-01,
                             1.951e-01]])]
       constr_nfev: [41808]
       constr_njev: [0]
       constr_nhev: [0]
                 v: [array([-8.121e+06])]
            method: tr_interior_point
        optimality: 7952820.982126532
  constr_violation: 0.0
    execution_time: 2097.7459330558777
         tr_radius: 9.073009965347647e-09
    constr_penalty: 1.0
 barrier_parameter

In [5]:
invariant(res.x)

prob = 0.008718522154469888
Invariant = 0.018273083776924434



0.018273083776924434

### Matriz de Knill

In [ ]:
V = np.array(
    [
        [-1 / 3, -sqrt(2) / 3, sqrt(2) / 3, 2 / 3],
        [sqrt(2) / 3, -1 / 3, -2 / 3, sqrt(2) / 3],
        [
            -sqrt(3 + sqrt(6)) / 3,
            sqrt(3 - sqrt(6)) / 3,
            -sqrt((3 + sqrt(6)) / 2) / 3,
            sqrt(1 / 6 - 1 / (3 * sqrt(6))),
        ],
        [
            -sqrt(3 - sqrt(6)) / 3,
            -sqrt(3 + sqrt(6)) / 3,
            -sqrt(1 / 6 - 1 / (3 * sqrt(6))),
            -sqrt((3 + sqrt(6)) / 2) / 3,
        ],
    ]
)